In [1]:
import keras
from keras.layers import Input, Dense, LSTM
from keras.models import Model, Sequential

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
filename = 'glove.6B.50d.txt'
def loadGloVe(filename):
    vocab = set()
    embd = {}
    word = None
    with open(filename) as f:
        try:
            for line in f:
                row = line.strip().split(' ')
                word = row[0]
                vocab.add(row[0])
                embd[word] = np.array([float(x) for x in row[1:]])
            
        except Exception as e:
            print(word)
    print('Loaded GloVe!')
    return vocab,embd
vocab_set,embd = loadGloVe(filename)
# vocab_size = len(vocab)
# vocab_set = set(vocab)
# embedding_dim = len(embd[0])
# embedding = np.asarray(embd)

Loaded GloVe!


In [4]:
vocab_set.add('<UNK>')

In [5]:
embd['<UNK>'] = np.zeros(50, dtype = np.float32)

In [6]:
vocab_list = list(vocab_set)
word_to_index = {vocab_list[i]: i for i in range(len(vocab_list))}

In [7]:
('<UNK>' in vocab_set)

True

In [9]:
# import json
# import os
# import numpy
# def generate_vocabulary():
#     i = 0
#     vocab = set() 
#     with open("tip.json",encoding="utf-8") as f:
#         for line in f:
#             tip = json.loads(line.strip())['text']
# #             print (tip)
#             words = tip.lower().split()
#             vocab |= set(words)
#             i +=1
#             if i == 5000:
#                 break
#     vocab.add('start')
    
#     return vocab,list(vocab)
# # def generate_data():
# vocab_set,vocab_list = generate_vocabulary()

In [8]:
# word_index = {vocab_list[i]:i for i in range(len(vocab_list))}
transnet_size = 64
sequence_length = 3
def one_hot(word,word_index,vocab_len):
    x = [0]*vocab_len
    if word in word_index:
        x[word_index[word]] = 1
    return x
# def one_hot_(k,l):
#     x = [0]*(l)
#     while k > 0:        
#         x[k%l] = 1
#         k = k//l
#     return x


In [11]:

# def generate_train():
#     j = 0
#     X = []
#     Y = []
#     total = 1000
#     tnet_file = open('../src/ex2_data/train_epochs/newfile.txt')
#     with open("./data/train_lex_data.txt") as f:
#         for line in f:
#             user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
#             #vec = one_hot_(j,transnet_size)
#             words = ['start' for _ in range(sequence_length)] + tip.strip().lower().split()
#             vectors = []
#             onehot = []
#             for word in words:
#                 encoding = one_hot(word)
#                 onehot += [encoding]
#                 vectors += [encoding+vec]
#             for i in range(sequence_length-1,len(words)-1):
#                 X += [vectors[i-sequence_length+1:i+1]]
#                 Y += [onehot[i+1]]
#             j += 1
#             if j % 100 == 0:
#                 print(j)
#             if j == total:
#                 break
#     return np.array(X),np.array(Y)
                    
                
            
            

In [12]:
# X,Y =generate_train()

In [10]:
import pickle
import string

In [11]:
punc = set(string.punctuation)


In [12]:
user_item_index = pickle.load(open('user_item_index_dict.pkl','rb'))

In [16]:
# user_item_index

In [14]:
import pickle
def new_generate_train():
    sequence_length = 3
    j = 0
    X = []
    Y = []
    total = 500
    tnet_file = pickle.load(open('./gen_review_vec_train.pkl','rb'))
    with open("./data/train.txt") as f:
        for line in f:
#             rev_emb = np.zeros(50)
            user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
            rev_index = user_item_index[(user_id,item_id)]
            
            #vec = one_hot_(j,transnet_size)
            rev_words = [w.lower() for w in tip.split()]
            words = []
            for w in rev_words:
                prev = 0
                for ch in range(len(w)):
                    if w[ch] in punc:
                        if prev < ch:
                            words += [w[prev:ch]]
                        words += [w[ch]]
                        prev = ch + 1
                if prev < len(w):
                    words += [w[prev:]]
            if len(words)< 5:
                words += ['<UNK>']*(5-len(words))
            elif len(words) > 5:
                words = words[:5]
            words = ['<UNK>' for _ in range(sequence_length)] + words
            vectors = []
            onehot = []
            vec = tnet_file[rev_index] #index of review from tnet file
            j += 1
#             if j % 10 == 0:
#                 print(j,end=" ")
            for word in words:
                if word in embd:
                    encoding = embd[word]
                else:
                    encoding = embd['<UNK>']
                onehot_encoding = one_hot(word,word_to_index,len(vocab_set))
                onehot += [onehot_encoding]
                vectors += [np.concatenate([encoding,vec])]
            X = []
            Y = []
            for i in range(sequence_length-1,len(words)-1):
                X += [vectors[i-sequence_length+1:i+1]]
                Y += [onehot[i+1]]
#             X,Y = np.array(X),np.array(Y)
#             print(X.shape,Y.shape)
            yield X,Y
#                 Y += []
            
#             if j == total:
#                 break
#     return X,Y#np.array(X),np.array(Y)
                    
                
            
            

In [15]:
X =[]
Y = []
for x,y in new_generate_train():
    X += x
    Y += y
X = np.array(X)
Y = np.array(Y)

In [16]:
X.shape

(2500, 3, 114)

In [17]:
Y.shape

(2500, 400001)

In [139]:
# i=0
# for x,y in new_generate_train():
#     if i % 10 == 0:
#         print(i,end=" ")
#     i += 1
#     if i == 600:
#         break

In [18]:
sequence_length = 3
transnet_size = 64
word_emb_size = 50

In [141]:
# keras.backend.clear_session()

In [19]:
V = len(vocab_list)
model = Sequential()
model.add(LSTM(15,input_shape=(sequence_length,word_emb_size+transnet_size)))
model.add(Dense(V,activation="softmax"))

In [20]:
len(vocab_list)

400001

In [21]:
# model.summary()
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])

In [145]:
# X_train, Y_train = a,b #generate_train()

In [22]:
import tensorflow as tf
# keras.backend.get_session().run(tf.global_variables_initializer())
model.fit(X,Y,nb_epoch=150)

/Users/Maverick/anaconda/lib/python2.7/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/20
2500/2500 [==============================] - 109s 44ms/step - loss: 9.0502 - acc: 0.0324
Epoch 2/20
2500/2500 [==============================] - 88s 35ms/step - loss: 6.3865 - acc: 0.0340
Epoch 3/20
2500/2500 [==============================] - 91s 37ms/step - loss: 6.0082 - acc: 0.0420
Epoch 4/20
2500/2500 [==============================] - 97s 39ms/step - loss: 5.7802 - acc: 0.0480
Epoch 5/20
2500/2500 [==============================] - 93s 37ms/step - loss: 5.5432 - acc: 0.0620
Epoch 6/20
2500/2500 [==============================] - 87s 35ms/step - loss: 5.3282 - acc: 0.0716
Epoch 7/20
2500/2500 [==============================] - 87s 35ms/step - loss: 5.1502 - acc: 0.0832
Epoch 8/20
2500/2500 [==============================] - 87s 35ms/step - loss: 5.0062 - acc: 0.0936
Epoch 9/20
2500/2500 [==============================] - 87s 35ms/step - loss: 4.8808 - acc: 0.1032
Epoch 10/20
2500/2500 [==============================] - 87s 35ms/step - loss: 4.7832 - acc: 0.0996
Epoch 11

In [23]:

#sample
import numpy as np
def long_to_small(preds):
    preds = preds.tolist()
    index = []
    small_preds = []
    for i in range(len(preds)):
        if preds[i] > 0.001:
            small_preds += [preds[i]]
            index += [i]
    return small_preds,index
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    #print preds[:10], np.max(preds)
    #print(len(preds))
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [70]:
# X_test,Y_test = generate_data(False)

In [71]:
# re  = model.predict(X_test)

In [72]:
# for i in range(len(re)):
#     print(vocab_list[sample(re[i])],end=" ")

In [24]:
out=[] 

In [27]:
import pickle
def test(out):
    sequence_length = 3
    
    j = 0
    X = []
    Y = []
    #total = 1000
    tnet_file = pickle.load(open('./gen_review_vec_test.pkl','rb')) #changr to test
    with open("./data/Old/test_lex_data.txt") as f:
        for line in f:
            rev_emb = np.zeros(50)
            user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
            #vec = one_hot_(j,transnet_size)
#             rev_words = [w.lower() for w in review_text.split()]
            words = []
            words = ['<UNK>' for _ in range(sequence_length)] + words
            vectors = []
            onehot = []
            vec = tnet_file[j]
            for word in words:
                onehot_encoding = one_hot(word,word_to_index,len(vocab_set))
                if word in embd:
                    encoding = embd[word]
                else:
                    encoding = embd['<UNK>']
                onehot += [onehot_encoding]
                vectors += [np.concatenate([encoding,vec])]
            #for i in range(sequence_length-1,len(words)-1):
            X += [vectors[0:sequence_length]]
#                 Y += [onehot[i+1]]
            j += 1
            yp = model.predict(np.array(X))[0]
            yp_small, index = long_to_small(yp)
            yp = vocab_list[index[sample(yp_small)]]
            generated_tip = [yp]
            words += [yp]
            for i in range(sequence_length,10+sequence_length):
                word = generated_tip[-1]
                onehot_encoding = one_hot(word,word_to_index,len(vocab_set))
                onehot += [onehot_encoding]
                if word in embd:
                    encoding = embd[word]
                else:
                    encoding = embd['<UNK>']
                vectors += [np.concatenate([encoding,vec])]
                X += [vectors[i-sequence_length+1:i+1]]
                x = [X[-1]]
                yp = model.predict(np.array(x))[0]
                yp_small, index = long_to_small(yp)
                yp = vocab_list[index[sample(yp_small)]]
                generated_tip += [yp]
                words += [yp]
            out += [" ".join(generated_tip)]
            if j == 5:
                break

In [28]:
test(out)

In [29]:
out

['a a really hair 9 for call . here . is',
 "for can a me . right . ' to sure in",
 '. prepared place good clean ! inside pizza around and have',
 'a with . been down ! ! spicy like salad for',
 "my the but ' s better ! will you were the"]

##### Old code

In [30]:
def generate_test_data():
    with open('tip.json') as f:
        i = 0
        for line in f:
            tip = json.loads(line.strip())['text']
            vec = one_hot_(i,transnet_size)
            X = []
            one_hot_v = one_hot('start')
            X = [one_hot_v+vec for _ in range(sequence_length)]
                
            return vec,tip,numpy.array(X).reshape(1,sequence_length,-1)

In [31]:
vec,tip,X = generate_test_data()

In [32]:
for i in range(10):
    y = model.predict(X)
#     print(y.shape)
#     print (y)
    w = sample(y[0])
#     print(w)
    print(vocab_list[w],end=" ")
    X = numpy.delete(X,0,1)
#     print(X.shape)
    X = numpy.insert(X,sequence_length-1,one_hot_(w,len(vocab_list))+vec,axis=1)
#     print(X.shape)
#     print(X[0][sequence_length-1])
# X.shape

the kidding. has even this up be people to easy 

In [33]:
tip

'Get here early enough to have dinner.'

In [ ]:
# import tensorflow as tf
# tf.__version__

In [78]:
pickle.load(open('./gen_review_vec_train.pkl','rb'),encoding="bytes")

array([[ 0.0677953 ,  0.12908284,  0.00995188, ...,  0.09868402,
         0.05035567,  0.0261237 ],
       [ 0.0677953 ,  0.12908284,  0.00995188, ...,  0.09868402,
         0.05035567,  0.0261237 ],
       [ 0.0677953 ,  0.12908284,  0.00995188, ...,  0.09868402,
         0.05035567,  0.0261237 ],
       ..., 
       [ 0.06633914,  0.1292228 ,  0.00890549, ...,  0.09720803,
         0.04496233,  0.02088072],
       [ 0.07077023,  0.13585889,  0.01115206, ...,  0.09501065,
         0.03852132,  0.01822961],
       [ 0.06911414,  0.13658382,  0.01111554, ...,  0.09772882,
         0.04012528,  0.02351698]], dtype=float32)

In [129]:
file = open ('data/train.txt','w',encoding='utf-8')
i = 0
with open("./data/train_lex_data.txt",encoding='utf-8') as f:
        for line    in f:
            user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
            if len(tip) < 25:
                continue
            file.write(line)
            i += 1
            if i == 500:
                break
file.close()